In [1]:
from abc import abstractmethod
from typing import Dict, List, Optional
from IPython.core.display import display
from ipywidgets import interact_manual
from ipywidgets.widgets.widget_int import IntSlider
from pandas import DataFrame

In [2]:
fb_groups = {
    "name": [
        "Pyrenean Mountaindog UK",
        "Pyrenean Mountaindog SE",
        "Dog walk group",
    ],
    "url": [
        "https://www.facebook.com/groups/111621008927356",
        "https://www.facebook.com/groups/1316148681731540",
        "https://www.facebook.com/groups/localgroupdogwalkenfieldandsurroundingareas",
    ],
    "joiners": [
        10,
        20,
        30
    ],
}

In [3]:
groups_df = DataFrame(fb_groups)

In [4]:
groups_df.head(3)

,name,url,joiners
0,Pyrenean Mountaindog UK,https://www.facebook.com/groups/111621008927356,10
1,Pyrenean Mountaindog SE,https://www.facebook.com/groups/1316148681731540,20
2,Dog walk group,https://www.facebook.com/groups/localgroupdogw...,30


In [5]:
min_joiners = IntSlider(min=1, max=groups_df["joiners"].max(), step=1, value=1)
max_joiners = IntSlider(min=1, max=groups_df["joiners"].max(), step=1, value=groups_df["joiners"].max())

In [6]:
@interact_manual
def display_plot(min_joiners=min_joiners, max_joiners=max_joiners):
    display(
        groups_df[
        (groups_df["joiners"] >= min_joiners)
        & (groups_df["joiners"] <= max_joiners)
    ].sort_values(by="joiners", ascending=False)
    )

interactive(children=(IntSlider(value=1, description='min_joiners', max=30, min=1), IntSlider(value=30, descri…

##### Since we want to hide the code we want to call the "display_plot" function independently
Like this?

```python
display_plot(min_joiners=min_joiners, max_joiners=max_joiners)
TypeError: '>=' not supported between instances of 'int' and 'IntSlider'
```
No.

Like this?

In [7]:
interact_manual(display_plot, min_joiners=min_joiners, max_joiners=max_joiners)

interactive(children=(IntSlider(value=15, description='min_joiners', max=30, min=1), IntSlider(value=30, descr…

<function __main__.display_plot(min_joiners=IntSlider(value=15, description='min_joiners', max=30, min=1), max_joiners=IntSlider(value=30, description='max_joiners', max=30, min=1))>

Maybe but you still have the magic of the dataframe not been explicitly passed as an argument.
With the function been hidden away in a module it won't be 100% transparent.
What would you do with the IntSliders? There are simply too many moving parts with this.
(or maybe I don't know what I'm doing so feel free to comment)


In [26]:
class AbstractDataclass:
    def __init__(self, dataframe: DataFrame, schema: Optional[Dict[str, any]]):
        self.dataframe = dataframe
        self.schema = schema
        self._validate()

    @abstractmethod
    def display(self, *args):
        return NotImplemented

    @property
    def _dataframe_columns(self) -> List[str]:
        return sorted(self.dataframe.columns)

    @property
    def _schema_fields(self) -> List[str]:
        return sorted(self.schema.keys())

    def _validate(self) -> None:
        if self._dataframe_columns != self._schema_fields:
            raise ValueError(f"dataframe columns names do not match.\n"
                             f"Want: {self._schema_fields},\n"
                             f"Got: {self._dataframe_columns}")

In [14]:
class Groups(AbstractDataclass):
    def __init__(self, dataframe: DataFrame):
        self.dataframe = dataframe
        self.schema = {
            "name": str,
            "url": str,
            "joiners": int,
        }
        super().__init__(dataframe, schema=self.schema)

    def display(self):
        @interact_manual(min_joiners=self.min_joiners, max_joiners=self.max_joiners)
        def _display(min_joiners: IntSlider, max_joiners: IntSlider):
            return self.dataframe[
                (self.dataframe["joiners"] > min_joiners)
                & (self.dataframe["joiners"] <= max_joiners)
            ].sort_values(by="joiners", ascending=False)

    @property
    def min_joiners(self) -> IntSlider:
        return IntSlider(min=1, max=groups_df["joiners"].max(), step=1, value=1)

    @property
    def max_joiners(self) -> IntSlider:
        return IntSlider(min=1, max=groups_df["joiners"].max(), step=1, value=groups_df["joiners"].max())

In [27]:
groups = Groups(dataframe=groups_df)
groups.display()

interactive(children=(IntSlider(value=1, description='min_joiners', max=30, min=1), IntSlider(value=30, descri…

#### One last thing about some basic validation

In [28]:
fb_groups_extra = {
    "name": [
        "Pyrenean Mountaindog UK",
        "Pyrenean Mountaindog SE",
        "Dog walk group",
    ],
    "url": [
        "https://www.facebook.com/groups/111621008927356",
        "https://www.facebook.com/groups/1316148681731540",
        "https://www.facebook.com/groups/localgroupdogwalkenfieldandsurroundingareas",
    ],
    "joiners": [
        10,
        20,
        30
    ],
    "leavers" : [
        1,
        2,
        3
    ],
}

In [29]:
fb_groups_extra_df = DataFrame(fb_groups_extra)

In [30]:
this_should_not_work = Groups(dataframe=fb_groups_extra_df)

ValueError: dataframe columns names do not match.
Want: ['joiners', 'name', 'url'],
Got: ['joiners', 'leavers', 'name', 'url']

In [ ]:
# from geopandas import read_file, datasets as geo_datasets
# import matplotlib.pyplot as plt
# fig, ax = plt.subplots()
# world = read_file(geo_datasets.get_path("naturalearth_lowres"))
# north_america = world.query('continent == "North America"')
# north_america.plot(ax=ax, alpha=0.2, color="#00296b", edgecolor="#faf6f6")
